In [1]:
!pip install lightning
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 45.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 546.0/546.0 kB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import lightning.pytorch as pl
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision.datasets import ImageFolder
from datasets import load_dataset
from transformers import DefaultDataCollator
from torchvision.models.vision_transformer import VisionTransformer
import torchvision
import seaborn as sns
from sklearn.model_selection import train_test_split
from tqdm import tqdm 
from PIL import Image

In [3]:
dataframe = pd.read_csv("/kaggle/input/early-detection-of-3d-printing-issues/train.csv")

In [4]:
root_dir = "/kaggle/input/early-detection-of-3d-printing-issues/"

# Split

For validation of the model, I have removed printer 22. For final training, I have kept printer 22 in the training set.

In [5]:
train_df, val_df = dataframe, dataframe[dataframe['printer_id'] == 22]

In [6]:
len(train_df), len(val_df)

(81060, 7492)

# Augumenting dataset and improving predictions using bagging

A neat trick to improve prediction is to have the model predict the class for 4 sub-images instead of a single image. What is a sub image? I have divided every image into 4 regions of size 240x320. This helps in keeping most of the image intact when downsampling to 200x200 and allows me to have a more accurate prediction.  

In [7]:
class ClassificationDataset(Dataset):

    def __init__(self, dataset):
        # Transforms
        self.dataset = dataset
        self.to_tensor = transforms.ToTensor()
        self.resize = transforms.Resize(size = (200, 200))
#         self.grayscale = transforms.Grayscale()
        self.root_dir = "/kaggle/input/early-detection-of-3d-printing-issues/images/"
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        
        row = self.dataset.iloc[idx]
        image = self.to_tensor(Image.open(self.root_dir + row['img_path']))
        image1 = self.resize(image[:, :240, :320])
        image2 = self.resize(image[:, 240:, :320])
        image3 = self.resize(image[:, :240, 320:])
        image4 = self.resize(image[:, 240:, 320:])
#         return self.grayscale(image), torch.Tensor([row['has_under_extrusion']])
        return torch.stack([image1, image2, image3, image4]), torch.stack([torch.Tensor([row['has_under_extrusion']])]*4)    

In [8]:
train_ds, val_ds = ClassificationDataset(train_df), ClassificationDataset(val_df)

In [9]:
train_dataloader = DataLoader(
    train_ds,
    batch_size=16,
    shuffle=True,
    pin_memory=True,
    num_workers=2
)
vl_dataloader = DataLoader(
    val_ds,
    batch_size=16,
    shuffle=True,
    pin_memory=True,
    num_workers=2
)

# Model

The model currently being used is the ResNet34 and ResNet50. Initial predictions were submitted on a randomly initialised ResNet34, now trying to run ResNet50 with trained weights. 

In [10]:
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()
        
    def forward(self, x):
        return x
    
class ResNetClassifier(pl.LightningModule):
    
    def __init__(self):
        super().__init__()
        
        self.resnet = torchvision.models.resnet101(weights="IMAGENET1K_V2")
        self.resnet.fc = nn.Linear(2048, 1)
        self.criterion = nn.BCEWithLogitsLoss()
    
    def forward(self, x):
        return self.resnet(x)
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.00001)
        # scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience = 5)
        
        return [optimizer]
    
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        x = x.reshape(-1, 3, 200, 200)
        y = y.reshape(-1, 1)
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
#         self.log("train_loss", loss.item(), prog_bar=True)
        return loss

    def predict_step(self, batch, batch_idx):
        x = batch
        x = x.reshape(-1, 3, 200, 200)
        y_hat = self(x)
        
        return y_hat
        
   

In [11]:
classifier = ResNetClassifier()

Downloading: "https://download.pytorch.org/models/resnet101-cd907fc2.pth" to /root/.cache/torch/hub/checkpoints/resnet101-cd907fc2.pth


  0%|          | 0.00/171M [00:00<?, ?B/s]

In [12]:
trainer = pl.Trainer(max_epochs = 10, accelerator = "gpu", enable_progress_bar = True, precision="16",)

INFO: Using 16bit None Automatic Mixed Precision (AMP)
INFO: GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs


In [13]:
trainer.fit(classifier, train_dataloader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name      | Type              | Params
------------------------------------------------
0 | resnet    | ResNet            | 42.5 M
1 | criterion | BCEWithLogitsLoss | 0     
------------------------------------------------
42.5 M    Trainable params
0         Non-trainable params
42.5 M    Total params
85.004    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=10` reached.


In [14]:
test_dataframe = pd.read_csv("/kaggle/input/early-detection-of-3d-printing-issues/test.csv")

In [15]:
class ClassificationDatasetTest(Dataset):

    def __init__(self, dataset):
        # Transforms
        self.dataset = dataset
        self.to_tensor = transforms.ToTensor()
        self.resize = transforms.Resize(size = (200, 200))
#         self.grayscale = transforms.Grayscale()
        self.root_dir = "/kaggle/input/early-detection-of-3d-printing-issues/images/"
        
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        
        row = self.dataset.iloc[idx]
        image = self.to_tensor(Image.open(self.root_dir + row['img_path']))
        image1 = self.resize(image[:, :240, :320])
        image2 = self.resize(image[:, 240:, :320])
        image3 = self.resize(image[:, :240, 320:])
        image4 = self.resize(image[:, 240:, 320:])
#         return self.grayscale(image), torch.Tensor([row['has_under_extrusion']])
        return torch.stack([image1, image2, image3, image4])   

In [16]:
test_dataloader = DataLoader(
    ClassificationDatasetTest(test_dataframe),
    batch_size=16,
    shuffle=False,
    pin_memory=True,
    num_workers=2
)

In [17]:
# trainer = pl.Trainer(max_epochs = 3, accelerator = "gpu", enable_progress_bar = True, precision="16",)
pred = trainer.predict(classifier, test_dataloader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 5067it [00:00, ?it/s]

In [18]:
pred = torch.cat(pred)

In [19]:
pred_ids = []
for i in range(0, pred.shape[0], 4):
    
    data = torch.mean(torch.sigmoid(pred[i:i+4].double())).item()
    pred_ids.append(int(data > 0.5))
    

In [20]:
test_dataframe['has_under_extrusion'] = pred_ids

In [21]:
test_dataframe[['img_path', 'has_under_extrusion']].to_csv("/kaggle/working/submission.csv", index=False)